# Mapping El Corrido de Gregorio 

### packages required {.unnumbered}

In [ ]:
import geopandas as gpd
import pandas as pd

import re

from geopy.geocoders import ArcGIS
import plotly.express as px 

import spacy
from spacy import displacy

### Extracting Name Enities (places) from text document {.unnumbered}

In [ ]:
text = "elcorridodegregoriocortez.txt"
with open(text, 'r', encoding='utf-8') as c:
    text = c.read()
    
def clean_text(text):
    cleaned= re.sub(r'[":;,.“”]', "", text)
    return(cleaned)
text = clean_text(text)
#print(text)


TxGPE=[]
nlp = spacy.load("tx_trained_ner")
doc =nlp(text) 
#print(doc)
for ent in doc.ents:
    #print(ent.text, ent.label_)
    if ent.label_ == "GPE":
        TxGPE.append(ent.text)
#print(TxGPE)

ents = [(e.text, e.start_char, e.end_char, e.label_)for e in doc.ents]
entsname=[(e.text) for e in doc.ents]
print(entsname)

### Visulization: Highlighting Name Entities from text {.unnumbered}

In [ ]:
displacy.render(doc, style ='ent', jupyter=True, page=True)

### Creating Dataframe {.unnumbered}

In [ ]:
df = pd.DataFrame({'NER': entsname})
geolocator = ArcGIS(user_agent='CorridosMap')
geocode = lambda query: geolocator.geocode("%s, Texas" % query)
#print(geocode(entsname))
df['Coordinates'] = df['NER'].apply(geocode)
df

### Creating GeoDataframe {.unnumbered}

In [ ]:
gdf = gpd.tools.geocode(df.Coordinates, provider='ArcGIS')
gdf = gpd.GeoDataFrame(gdf, crs="EPSG:4326")
gdf["lat"]=gdf['geometry'].y
gdf ["lon"] = gdf['geometry'].x

gdf
gdf.to_json()

### Creating Mapping Visualization from El Corrido de Gregorio Cortez {.unnumbered}

In [ ]:
px.set_mapbox_access_token(open("mapboxtoken").read())
fig = px.line_mapbox(gdf,
                        lat=gdf.geometry.y,
                        lon=gdf.geometry.x,
                        hover_name="address",
            
                        )

fig.update_layout(mapbox_style="open-street-map", mapbox_zoom=6, mapbox_center_lat = 29,
    margin={"r":0,"t":0,"l":0,"b":0})

fig.show()